In [2]:


import pyarrow.csv as pv
import pyarrow.parquet as pq
import os
import pandas as pd
# 定义输入和输出文件夹
input_folder = "data2"  # CSV.gz 文件所在目录
output_folder = "data"  # Parquet 存放目录

# 确保输出目录存在
# os.makedirs(output_folder, exist_ok=True)

# # # 遍历 data2 目录下所有 .csv.gz 文件
# for file_name in os.listdir(input_folder):
#     if file_name.endswith(".csv.gz"):  # 仅处理 CSV.gz 文件
#         input_path = os.path.join(input_folder, file_name)
#         output_path = os.path.join(output_folder, file_name.replace(".csv.gz", ".parquet"))

#         print(f"正在转换: {input_path} -> {output_path}")
        
#         # 读取 CSV.gz 为 PyArrow 表（不会一次性加载到内存）
#         table = pv.read_csv(input_path)
        
#         # 保存为 Parquet
#         pq.write_table(table, output_path)

# print("所有文件已转换完成")




def _parse_slice(s):
    """Convert int/slice string to proper slice object"""
    if isinstance(s, int):
        return slice(s, s+1)  # Single item slice
    elif isinstance(s, str):
        if ':' in s:
            parts = [int(x) if x else None for x in s.split(':')]
            return slice(*parts)
        else:
            return slice(int(s), int(s)+1)
    return s

def investigate(table, row=None, col=None):
    """
    Analyze parquet file with flexible slicing:
    - row/col can be int (single item) or slice spec (e.g., '2:5')
    - Returns: (submatrix, metadata) where metadata is (colnames, nrow, ncol)
    
    Example usage:
    investigate("data.parquet", row=1, col=1)  # 2nd row & 2nd column
    investigate("data.parquet", row="1:4")     # rows 2-4 (Python slice)
    """
    # Read entire table
    nrow, ncol = table.num_rows, table.num_columns
    
    # Parse slicing parameters
    row_slice = _parse_slice(row) if row is not None else slice(None)
    col_slice = _parse_slice(col) if col is not None else slice(None)
    
    # Handle column selection by index/name
    if isinstance(col_slice, int):
        col_idx = col_slice
        if col_idx >= ncol:
            raise ValueError(f"Column index {col_idx} out of range ({ncol} cols)")
        col_names = [table.column_names[col_idx]]
    else:
        col_names = table.column_names[col_slice]
    
    # Slice the table
    sub_table = table.slice(row_slice.start, row_slice.stop).select(col_names)
    
    # Convert to pandas for display
    df = sub_table.to_pandas()
    
    # Prepare metadata
    metadata = (table.column_names, nrow, ncol)
    
    return df, metadata








In [3]:
#这个模块是读取数据的

lab = pq.read_table('data/lab.parquet')
apachePatientResult=pq.read_table('data/apachePatientResult.parquet')

apacheApsVar = pq.read_table('data/apacheApsVar.parquet')

patient = pq.read_table('data/patient.parquet')
# nurseAssessment = pq.read_table('data/nurseAssessment.parquet')

In [4]:
import gc
gc.collect()  # 手动清理内存

0

patient  unitdischargeoffset ->Y
一个patient进入icu次数  unitvisitnumber

sample_n
patient: subclass desease apacheadmissiondx挑选病理种类得到subsample

hospital id, give subsample*****
X selection:
patient age, gender, ethnicity人种,admissionweight,dischargeheight,

vitalperidic, 第一次测量的各种值

lab
apachpatientresult: apachescore,select apachversion IVa, predictedicumortality,predictediculos

一个病人会有很多病，选择第一次进ICU的病,在patient中有.



In [5]:
#读取关键数据
dfpatient,metapatient=investigate(patient,row="0:3000000")
dflab,metalab=investigate(lab,row="0:40000000")
dfAPR,metaAPR=investigate(apachePatientResult,row="0:3000000")
most_frequent_id = dfpatient["hospitalid"].value_counts().idxmax()

# 提取同一 hospitalid 的数据


In [6]:
dfpatient.hospitalid

0          59
1          60
2          60
3          73
4          67
         ... 
200854    458
200855    458
200856    458
200857    459
200858    458
Name: hospitalid, Length: 200859, dtype: int64

In [7]:
#选择医院代号188,不可在不同医院之间预测ICU次数

dfpatient1 = dfpatient[dfpatient["hospitalid"] == 188]
dfpatient1 = dfpatient1.dropna(subset=["admissionheight"])
dfpatient1 = dfpatient1.dropna(subset=["admissionweight"])
dfpatient1

,patientunitstayid,patienthealthsystemstayid,gender,age,ethnicity,hospitalid,wardid,apacheadmissiondx,admissionheight,hospitaladmittime24,...,unitadmitsource,unitvisitnumber,unitstaytype,admissionweight,dischargeweight,unitdischargetime24,unitdischargeoffset,unitdischargelocation,unitdischargestatus,uniquepid
63630,980543,723071,Male,62,Caucasian,188,434,"ARDS-adult respiratory distress syndrome, non-...",182.0,02:01:00,...,Other Hospital,1,admit,101.3,93.3,18:47:00,3883,Floor,Alive,008-19039
63631,980545,723072,Female,> 89,Caucasian,188,434,"Sepsis, GI",151.0,06:02:00,...,Emergency Department,2,transfer,38.1,NaN,20:54:00,3575,Death,Expired,008-15867
63632,980561,723085,Male,57,Caucasian,188,445,"Sepsis, other",176.0,01:02:00,...,Emergency Department,2,transfer,77.0,77.0,17:53:00,3478,Death,Expired,008-57888
63633,980570,723091,Female,30,Caucasian,188,434,Coma/change in level of consciousness (for hep...,147.0,23:39:00,...,Emergency Department,2,transfer,94.3,94.3,00:32:00,1405,Step-Down Unit (SDU),Alive,008-33198
63634,980572,723093,Male,71,Caucasian,188,451,"CVA, cerebrovascular accident/stroke",172.0,19:04:00,...,Emergency Department,1,admit,81.3,NaN,00:09:00,1727,Step-Down Unit (SDU),Alive,008-18493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67226,1055371,782046,Male,68,Caucasian,188,464,"Tamponade, pericardial",180.3,03:20:00,...,Other Hospital,1,admit,80.9,NaN,20:32:00,3903,Floor,Alive,008-28756
67227,1055377,782051,Female,87,Other/Unknown,188,464,"Infarction, acute myocardial (MI)",137.0,10:20:00,...,Emergency Department,1,admit,38.9,39.7,18:54:00,1927,Step-Down Unit (SDU),Alive,008-9759
67228,1055396,782066,Female,75,African American,188,451,"CVA, cerebrovascular accident/stroke",157.0,05:01:00,...,Floor,1,admit,85.9,NaN,05:29:00,1479,Other,Alive,008-4701
67229,1055410,782077,Male,67,African American,188,434,"Pneumonia, other",172.2,21:37:00,...,Emergency Department,2,transfer,64.3,70.7,23:33:00,7200,Floor,Alive,008-52565


In [8]:
#clean lab and APR table
df_pivot_lab = dflab.pivot_table(index="patientunitstayid", columns="labname", values="labresult", aggfunc="first").reset_index()

def clean_df(df, K=20):

    # 确保存在 'patientunitstayid' 列
    if 'patientunitstayid' not in df.columns:
        raise ValueError("DataFrame 必须包含 'patientunitstayid' 列")

    # 排除 'patientunitstayid'，计算其他列的 NA 数量
    columns_to_consider = df.columns.drop('patientunitstayid')
    if len(columns_to_consider) < K:
        raise ValueError(f"可用列数 ({len(columns_to_consider)}) 不足以选择 K={K} 列")

    # 计算各列的 NA 数量并排序
    na_counts = df[columns_to_consider].isna().sum()
    selected_columns = na_counts.nsmallest(K).index.tolist()

    # 构建新 DataFrame
    cleaned_df = df[['patientunitstayid'] + selected_columns]

    # 过滤选中的 K 列中所有 NA 的行
    cleaned_df = cleaned_df.dropna(subset=selected_columns)

    return cleaned_df
df_pivotlab_clean=clean_df(df_pivot_lab,K=20)




filtered_df = dfAPR[dfAPR["apacheversion"] == "IVa"]
lsit_columns=['predictedicumortality',
       'predictediculos', 'actualiculos', 'predictedhospitalmortality', 'predictedhospitallos', 'actualhospitallos']


df_APR_clean=filtered_df[['patientunitstayid'] + lsit_columns]
df_APR_clean

,patientunitstayid,predictedicumortality,predictediculos,actualiculos,predictedhospitalmortality,predictedhospitallos,actualhospitallos
1,141168,0.028889,3.091127,2.4972,0.059099,6.628720,2.4972
3,141194,0.046448,4.167129,3.3423,0.102283,12.978228,9.2167
4,141203,0.291609,8.670299,1.2979,0.470973,16.319389,3.7493
6,141208,0.001330,0.444563,0.5000,0.002249,0.521041,0.4215
8,141227,0.335934,8.209624,1.1472,0.488562,16.508280,1.8861
...,...,...,...,...,...,...,...
297055,3353235,0.011009,1.934817,0.7423,0.022630,5.759009,2.6701
297057,3353237,0.026984,2.566297,0.8812,0.062568,7.565244,6.7215
297059,3353251,0.876797,8.920949,11.2909,0.918169,11.222566,13.4056
297061,3353254,0.011004,2.045046,0.2993,0.025801,6.199350,4.4549


In [9]:
#得到最终的数据表格，但未完成category factor的转化
from collections import defaultdict, Counter
import nltk
from nltk.corpus import wordnet
import pandas as pd
nltk.download('averaged_perceptron_tagger_eng')
# Merge dfpatient and df_pivot_clean on 'patientunitstayid'
merged_df = dfpatient1.merge(df_pivotlab_clean, on='patientunitstayid', how='inner')

# Merge the result with df_APR_clean on 'patientunitstayid'
final_df = merged_df.merge(df_APR_clean, on='patientunitstayid', how='inner')

# Display the final merged dataframe
final_df['apacheadmissiondx']

# final_df
final_df

# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')
final_df.columns
selected_column=['patientunitstayid', 'unitvisitnumber',  'gender', 'age',
       'ethnicity',  'apacheadmissiondx',
       'admissionheight', 
        'unitadmitsource', 
       'admissionweight', 'sodium', 'creatinine', 'BUN', 'potassium', 'Hct',
       'glucose', 'chloride', 'WBC x 1000', 'Hgb', 'RBC', 'calcium',
       'platelets x 1000', 'MCV', 'MCHC', 'bicarbonate', 'RDW', 'MCH',
       'anion gap', '-lymphs', '-monos', 'predictedicumortality',
       'predictediculos', 'actualiculos', 'predictedhospitalmortality',
       'predictedhospitallos', 'actualhospitallos']

df=final_df[selected_column]
df

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/xuranm/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


,patientunitstayid,unitvisitnumber,gender,age,ethnicity,apacheadmissiondx,admissionheight,unitadmitsource,admissionweight,sodium,...,MCH,anion gap,-lymphs,-monos,predictedicumortality,predictediculos,actualiculos,predictedhospitalmortality,predictedhospitallos,actualhospitallos
0,980543,1,Male,62,Caucasian,"ARDS-adult respiratory distress syndrome, non-...",182.0,Other Hospital,101.3,132.0,...,27.8,7.0,2.0,3.6,0.243194,9.369929,2.6965,0.345213,16.128035,8.6250
1,980545,2,Female,> 89,Caucasian,"Sepsis, GI",151.0,Emergency Department,38.1,142.0,...,25.7,11.0,8.6,3.9,0.087053,4.145974,2.4826,0.184933,11.575444,2.7063
2,980561,2,Male,57,Caucasian,"Sepsis, other",176.0,Emergency Department,77.0,140.0,...,30.3,22.0,13.3,6.1,0.750429,10.093950,2.4152,0.803781,16.542953,2.7799
3,980570,2,Female,30,Caucasian,Coma/change in level of consciousness (for hep...,147.0,Emergency Department,94.3,142.0,...,33.4,12.0,22.2,4.2,0.033486,4.886029,0.9756,0.061963,10.141401,5.7514
4,980572,1,Male,71,Caucasian,"CVA, cerebrovascular accident/stroke",172.0,Emergency Department,81.3,136.0,...,31.4,9.0,28.7,5.4,0.042978,2.874620,1.1993,0.107104,8.020842,1.8819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2627,1055371,1,Male,68,Caucasian,"Tamponade, pericardial",180.3,Other Hospital,80.9,135.0,...,31.0,6.0,13.4,5.3,0.014983,3.032703,2.7104,0.038153,9.211632,4.8542
2628,1055377,1,Female,87,Other/Unknown,"Infarction, acute myocardial (MI)",137.0,Emergency Department,38.9,134.0,...,28.0,10.0,28.4,5.3,0.054522,2.659347,1.3381,0.098785,5.745960,2.6340
2629,1055396,1,Female,75,African American,"CVA, cerebrovascular accident/stroke",157.0,Floor,85.9,147.0,...,30.5,20.0,6.6,3.4,0.705534,8.756908,1.0270,0.836033,16.307568,2.1444
2630,1055410,2,Male,67,African American,"Pneumonia, other",172.2,Emergency Department,64.3,131.0,...,32.2,6.0,2.0,6.0,0.260032,8.394489,5.0000,0.376197,14.641438,20.0181


In [10]:

df['apacheadmissiondx']
num_unique = df['apacheadmissiondx'].nunique()
num_unique

word_list = []
for row in df['apacheadmissiondx']:
    if pd.notna(row):  # 忽略 NaN 值
        words = row.split()  # 按空格拆分单词（自动过滤空字符串）
        # 去重并保持原始顺序
        seen = set()
        unique_words = [word for word in words if not (word in seen or seen.add(word))]
        word_list.extend(unique_words)

# 词性标注
tagged_words = nltk.pos_tag(word_list)

# 过滤出名词和形容词
nouns_and_adjs = [
    word for word, tag in tagged_words
    if tag.startswith('NN') or tag.startswith('JJ')
]

# 统计频率
word_freq = Counter(nouns_and_adjs)
word_freq

K = 60

# 筛选频率大于 K 的单词
filtered_word_freq = {word: freq for word, freq in word_freq.items() if freq > K}

filtered_word_freq


def filter_same_freq_words(filtered_word_freq, df):
    # 按频率分组
    freq_to_words = defaultdict(list)
    for word, freq in filtered_word_freq.items():
        freq_to_words[freq].append(word)

    # 记录需要保留的单词
    words_to_keep = set()

    # 检查每组频率相同的单词是否总是出现在相同的行中
    for freq, words in freq_to_words.items():
        if len(words) >= 1:  # 只检查频率相同的多个单词
            # 创建单词到行索引的映射
            word_to_rows = defaultdict(set)
            for idx, row in df['apacheadmissiondx'].items():
                if pd.notna(row):
                    row_words = row.split()
                    for word in words:
                        if word in row_words:
                            word_to_rows[word].add(idx)
            # 检查所有单词的行索引是否相同
            first_word_rows = word_to_rows[words[0]]
            all_same = True
            for word in words:
                if word_to_rows[word] != first_word_rows:
                    all_same = False
                    break
            # 如果所有单词的行索引相同，则只保留第一个单词
            if all_same:
                words_to_keep.add(words[0])
            else:
                words_to_keep.update(words)
        else:  # 频率唯一的单词直接保留
            words_to_keep.add(words[0])

    # 构建结果字典，只保留需要保留的单词
    result = {word: freq for word, freq in filtered_word_freq.items() if word in words_to_keep}
    return result




dictionary=filter_same_freq_words(filtered_word_freq,df)
dictionary

{'respiratory': 179,
 'pulmonary': 109,
 'Sepsis,': 330,
 'GI': 179,
 'other': 574,
 '(for': 71,
 'see': 181,
 'CVA,': 180,
 'cerebrovascular': 204,
 'Respiratory': 388,
 'medical,': 319,
 'Diabetic': 93,
 'ketoacidosis': 81,
 'Hemorrhage/hematoma,': 64,
 'Infarction,': 134,
 'acute': 96,
 'unknown': 143,
 'Pneumonia,': 124,
 'Cardiac': 137,
 '(with': 140,
 'System)': 161,
 'Bleeding,': 170,
 'upper': 70,
 'CABG': 106,
 'Rhythm': 86,
 'disturbance': 105,
 'coronary': 92,
 'Cardiovascular': 84,
 'alone,': 70}

In [11]:
import re

def contains_word(a, b):
    # Match the word `a` even if surrounded by punctuation/whitespace
    pattern = rf'(?<!\w){re.escape(a)}(?!\w)'  # More robust check
    return bool(re.search(pattern, b))


df_new = df.copy()
word_list=list(dictionary.keys())
word_length = len(word_list)
vector_length = word_length + 1
apache_vectors = []
for row in df['apacheadmissiondx']:
    # print(row)
    vector = [0] * vector_length
    for word in word_list:
        if contains_word(word,row):
            vector[word_list.index(word)]=1
    apache_vectors.append(vector)

# 添加到新的 DataFrame
for i in range(vector_length):
    column_name = f"{'apacheadmissiondx'}_{i}"
    df_new[column_name] = [v[i] for v in apache_vectors]

# 删除原始的 apacheadmissiondx 列
df_new.drop('apacheadmissiondx', axis=1, inplace=True)


In [12]:
import re

def contains_word(a, b):
    pattern = rf'\b{re.escape(a)}\b'
    return bool(re.search(pattern, b))


def one_hot_encode_categorical(df, category_column_name, word_list):
    df_new = df.copy()

    for column in category_column_name:
        if column == 'apacheadmissiondx':
            # 处理 apacheadmissiondx 列
            word_length = len(word_list)
            vector_length = word_length + 1

            # 生成对应的向量
            apache_vectors = []
            for row in df[column]:
                vector = [0] * vector_length
                
                apache_vectors.append(vector)

            # 添加到新的 DataFrame
            for i in range(vector_length):
                column_name = f"{column}_{i}"
                df_new[column_name] = [v[i] for v in apache_vectors]

            # 删除原始的 apacheadmissiondx 列
            df_new.drop(column, axis=1, inplace=True)
        else:
            # 处理普通列，转换为独热编码
            unique_values = df[column].unique()
            for value in unique_values:
                column_name = f"{column}_{value}"
                df_new[column_name] = (df[column] == value).astype(int)
            # 删除原始的列
            df_new.drop(column, axis=1, inplace=True)

    return df_new



['patientunitstayid', 'unitvisitnumber',  'gender', 'age',
       'ethnicity',  'apacheadmissiondx',
       'admissionheight', 
        'unitadmitsource', 
       'admissionweight', 'sodium', 'creatinine', 'BUN', 'potassium', 'Hct',
       'glucose', 'chloride', 'WBC x 1000', 'Hgb', 'RBC', 'calcium',
       'platelets x 1000', 'MCV', 'MCHC', 'bicarbonate', 'RDW', 'MCH',
       'anion gap', '-lymphs', '-monos', 'predictedicumortality',
       'predictediculos', 'actualiculos', 'predictedhospitalmortality',
       'predictedhospitallos', 'actualhospitallos']

dfnew=one_hot_encode_categorical(df,['gender', 'ethnicity','unitadmitsource'],dictionary)
dfnew.apacheadmissiondx.to_csv("1.csv")
dfnew.loc[dfnew["age"] =='> 89', "age"] = 89
dfnew.to_csv("output.csv", index=False, encoding="utf-8")

In [13]:
dfnew.loc[dfnew["age"] =='> 89', "age"] = 89
dfnew.to_csv("output.csv", index=False, encoding="utf-8")

In [14]:

# Define category mapping
category_map = {
    "ARDS": "Respiratory",
    "pneumonia": "Respiratory",
    "post-extubation edema": "Respiratory",
    "bacteremia": "Sepsis-related",
    "infection": "Sepsis-related",
    "hemoptysis": "Respiratory",
    "tracheostomy": "Respiratory",
    "Cardiac arrest": "Cardiac",
    "CHF": "Cardiac",
    "Infarction": "Cardiac",
    "hypertension": "Cardiac",
    "stroke": "Neurological",
    "Sepsis": "Sepsis-related",
    "Septic shock": "Sepsis-related",
    "CVA": "Neurological",
    "coma": "Neurological",
    "intracranial hemorrhage": "Neurological",
    "seizure": "Neurological",
    "subarachnoid hemorrhage": "Neurological",
    "GI bleeding": "Gastrointestinal",
    "peritonitis": "Gastrointestinal",
    "variceal bleeding": "Gastrointestinal",
    "surgery": "Gastrointestinal",
    "Acid-base": "Metabolic/Electrolyte",
    "electrolyte": "Metabolic/Electrolyte",
    "diabetes": "Metabolic/Electrolyte",
    "cocaine": "Toxicology",
    "benzodiazepines": "Toxicology",
    "calcium channel blockers": "Toxicology",
    "analgesic": "Toxicology",
    "trauma": "Surgical/Trauma",
    "transphenoidal surgery": "Surgical/Trauma"
}

# Function to categorize each diagnosis
def categorize_diagnosis(diagnosis_str):
    diagnoses = diagnosis_str.split(", ")  # Split by comma
    category_set = set()  # Use a set to avoid duplicates
    
    for diagnosis in diagnoses:
        for key, category in category_map.items():
            if key.lower() in diagnosis.lower():  # Case-insensitive matching
                category_set.add(category)
    
    return ", ".join(category_set) if category_set else "Other"  # Combine categories

# Apply function to create the new categorized column
dfnew["apacheadmissiondx_category"] = dfnew["apacheadmissiondx"].apply(categorize_diagnosis)

# Save to a new CSV
dfnew.drop('apacheadmissiondx', axis=1, inplace=True)


In [15]:
df_category_vectors = dfnew["apacheadmissiondx_category"].str.get_dummies(sep=", ")
df_combined = pd.concat([dfnew, df_category_vectors], axis=1)

# Drop the original apacheadmissiondx_category column
df_combined = df_combined.drop("apacheadmissiondx_category", axis=1)

# Save the final transformed DataFrame
df_combined.to_csv("final_transformed_data_188.csv", index=False)


In [16]:
df_combined

,patientunitstayid,unitvisitnumber,age,admissionheight,admissionweight,sodium,creatinine,BUN,potassium,Hct,...,unitadmitsource_,Cardiac,Gastrointestinal,Metabolic/Electrolyte,Neurological,Other,Respiratory,Sepsis-related,Surgical/Trauma,Toxicology
0,980543,1,62,182.0,101.3,132.0,0.90,33.0,4.8,29.7,...,0,0,0,0,0,0,1,0,0,0
1,980545,2,89,151.0,38.1,142.0,3.10,52.0,4.1,23.3,...,0,0,0,0,0,0,0,1,0,0
2,980561,2,57,176.0,77.0,140.0,3.00,24.0,4.3,27.1,...,0,0,0,0,0,0,0,1,0,0
3,980570,2,30,147.0,94.3,142.0,2.10,20.0,4.8,33.5,...,0,1,0,0,1,0,0,0,0,0
4,980572,1,71,172.0,81.3,136.0,1.30,16.0,4.3,39.7,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2627,1055371,1,68,180.3,80.9,135.0,1.00,17.0,3.6,42.6,...,0,0,0,0,0,1,0,0,0,0
2628,1055377,1,87,137.0,38.9,134.0,0.99,15.0,4.5,38.2,...,0,1,0,0,0,0,0,0,0,0
2629,1055396,1,75,157.0,85.9,147.0,1.90,33.0,4.0,38.1,...,0,0,0,0,1,0,0,0,0,0
2630,1055410,2,67,172.2,64.3,131.0,0.59,16.0,3.8,35.5,...,0,0,0,0,0,0,1,0,0,0


In [17]:
sum(df_combined.unitvisitnumber==7)

2